In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

tickers = ['AAPL', 'MSFT', 'GOOGL']
start_date = '2010-01-01'
end_date = '2020-12-31'
initial_funds = 100000
portfolio = {ticker: {'shares': 0, 'data': None} for ticker in tickers}
funds_per_stock = initial_funds / len(tickers)

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    data['Future_Return'] = data['Close'].pct_change().shift(-1)
    data.dropna(inplace=True)
    X = data[['SMA_50', 'SMA_200']]
    y = np.where(data['Future_Return'] > 0, 1, 0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    data['Predictions'] = model.predict(X[['SMA_50', 'SMA_200']])
    data['Signal'] = np.where(data['Predictions'] > 0, 1, -1)
    portfolio[ticker]['data'] = data

portfolio_value = pd.DataFrame(index=data.index)
portfolio_value['Value'] = 0

for date in portfolio_value.index:
    daily_value = 0
    for ticker in tickers:
        data = portfolio[ticker]['data']
        if date in data.index:
            if portfolio[ticker]['shares'] == 0 and funds_per_stock > 0:
                portfolio[ticker]['shares'] = funds_per_stock / data.loc[date, 'Close']
            daily_value += portfolio[ticker]['shares'] * data.loc[date, 'Close']
    portfolio_value.loc[date, 'Value'] = daily_value

total_return = portfolio_value['Value'].iloc[-1] - initial_funds
mean_return = portfolio_value['Value'].pct_change().mean()
volatility = portfolio_value['Value'].pct_change().std()

print(f"Total Return: {total_return}")
print(f"Mean Daily Return: {mean_return}")
print(f"Volatility: {volatility}")


In [3]:
def download_and_prepare_data(tickers, start_date, end_date):
    all_data = {}
    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date)
        data.columns = data.columns.str.lower()  # Ensure column names are lowercase
        
        # Calculate technical indicators
        data['rsi'] = rsi(data)
        data['macd'], data['macdsignal'], data['macdhist'] = macd(data)
        data['k'], data['d'] = stochastic_oscillator(data)
        bollinger_bands(data)
        data.dropna(inplace=True)  # Remove rows with missing values
        
        # Store the prepared data
        all_data[ticker] = data
    return all_data


In [4]:
def generate_signals(data):
    for ticker, df in data.items():
        df['signal'] = np.where(df['close'] > df['close'].rolling(window=20).mean(), 1, 0)  # Example strategy
    return data


In [5]:
def simulate_trades(all_data, initial_funds):
    portfolio_value = initial_funds
    holdings = {ticker: 0 for ticker in all_data}  # Initial holdings
    
    # Collect daily portfolio value for performance analysis
    portfolio_history = []
    
    # Assuming all_data has the same date range for simplicity
    dates = all_data[next(iter(all_data))].index  # Get dates from the first stock's data
    
    for date in dates:
        for ticker, df in all_data.items():
            if date in df.index:  # Check if the date exists in the stock data
                data_row = df.loc[date]
                signal = data_row['signal']
                close_price = data_row['close']
                
                # Implement your trading strategy here
                # This is a simple example: buy if signal is 1, sell if signal is 0
                if signal == 1 and portfolio_value >= close_price:  # Buy
                    shares_to_buy = portfolio_value // close_price
                    holdings[ticker] += shares_to_buy
                    portfolio_value -= shares_to_buy * close_price
                elif signal == 0 and holdings[ticker] > 0:  # Sell
                    portfolio_value += holdings[ticker] * close_price
                    holdings[ticker] = 0
        
        # Update portfolio history
        total_value = portfolio_value + sum(holdings[ticker] * df.loc[date, 'close'] for ticker, df in all_data.items() if date in df.index)
        portfolio_history.append([date, total_value])
    
    portfolio_history_df = pd.DataFrame(portfolio_history, columns=['Date', 'Total Value'])

    return portfolio_history_df


In [6]:
tickers = ['AAPL', 'MSFT', 'GOOGL']  # Define your portfolio
start_date = '2010-01-01'
end_date = '2021-12-31'
initial_funds = 55555  # Starting with $100,000

# Download and prepare data
all_data = download_and_prepare_data(tickers, start_date, end_date)

# Generate buy/sell signals
all_data = generate_signals(all_data)

# Simulate trades
portfolio_history_df = simulate_trades(all_data, initial_funds)

# Print final portfolio value
final_portfolio_value = portfolio_history_df['Total Value'].iloc[-1]
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")


/Users/ritenbhagra/miniconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/ritenbhagra/miniconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/ritenbhagra/miniconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[***

Final Portfolio Value: $689817.11


In [7]:

portfolio_history_df.set_index('Date', inplace=True)
final_value = portfolio_history_df['Total Value'].iloc[-1]

total_days = len(portfolio_history_df)
years = total_days / 252
annual_return = ((final_value / initial_funds) ** (1 / years)) - 1
daily_returns = portfolio_history_df['Total Value'].pct_change().dropna()
sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)

print(f"Final Portfolio Value: ${final_value:.2f}")
print(f"Annualized Return: {annual_return * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


Final Portfolio Value: $689817.11
Annualized Return: 23.68%
Sharpe Ratio: 1.07


In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

tickers = ['AAPL', 'MSFT', 'GOOGL']
start_date = '2010-01-01'
end_date = '2020-12-31'
initial_funds = 100000
portfolio = {ticker: {'shares': 0, 'data': None} for ticker in tickers}
funds_per_stock = initial_funds / len(tickers)

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    data['Future_Return'] = data['Close'].pct_change().shift(-1)
    data.dropna(inplace=True)
    X = data[['SMA_50', 'SMA_200']]
    y = np.where(data['Future_Return'] > 0, 1, 0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    data['Predictions'] = model.predict(X[['SMA_50', 'SMA_200']])
    data['Signal'] = np.where(data['Predictions'] > 0, 1, -1)
    portfolio[ticker]['data'] = data

portfolio_value = pd.DataFrame(index=data.index)
portfolio_value['Value'] = 0

for date in portfolio_value.index:
    daily_value = 0
    for ticker in tickers:
        data = portfolio[ticker]['data']
        if date in data.index:
            if portfolio[ticker]['shares'] == 0 and funds_per_stock > 0:
                portfolio[ticker]['shares'] = funds_per_stock / data.loc[date, 'Close']
            daily_value += portfolio[ticker]['shares'] * data.loc[date, 'Close']
    portfolio_value.loc[date, 'Value'] = daily_value

total_return = portfolio_value['Value'].iloc[-1] - initial_funds
mean_return = portfolio_value['Value'].pct_change().mean()
volatility = portfolio_value['Value'].pct_change().std()
annualized_return = (1 + mean_return) ** 252 - 1
sharpe_ratio = (mean_return / volatility) * np.sqrt(252)

print(f"Final Portfolio Value: {portfolio_value['Value'].iloc[-1]}")
print(f"Total Return: {total_return}")
print(f"Annualized Return: {annual_return * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio}")


/Users/ritenbhagra/miniconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/ritenbhagra/miniconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/ritenbhagra/miniconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[***

Final Portfolio Value: 874737.4268801396
Total Return: 774737.4268801396
Annualized Return: 23.68%
Sharpe Ratio: 1.0568236022681128
